In [0]:
%sql
ALTER TABLE dev.bronze.sales
SET TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [0]:
SELECT * FROM dev.bronze.sales

In [0]:
UPDATE dev.bronze.sales
SET Description = "test"
WHERE StockCode == "84879"

In [0]:
DESCRIBE HISTORY dev.bronze.sales

In [0]:
SELECT * FROM table_changes("dev.bronze.sales", 5)

In [0]:
DROP TABLE dev.bronze.metadata

In [0]:
CREATE TABLE IF NOT EXISTS dev.bronze.metadata(
  metadata_key BIGINT GENERATED ALWAYS AS IDENTITY,
  table_name STRING,
  version_processed INT,
  created_time TIMESTAMP
);

In [0]:
SELECT COUNT(1), COUNT(concat(InvoiceNo, StockCode)) FROM dev.bronze.sales

In [0]:
SELECT * FROM delta.`/Volumes/dev/bronze/sales_vol/delta/`

In [0]:
ALTER TABLE delta.`/Volumes/dev/bronze/sales_vol/delta/`
SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
DESCRIBE HISTORY delta.`/Volumes/dev/bronze/sales_vol/delta/`
-- SELECT
--   *
-- FROM
--   table_changes(
--     "delta.`/Volumes/dev/bronze/sales_vol/delta/`",
--     1
--   )

In [0]:
CREATE SCHEMA IF NOT EXISTS dev.silver;
CREATE TABLE IF NOT EXISTS dev.silver.sales
AS
SELECT * FROM dev.bronze.sales WHERE 1=2

In [0]:
DESCRIBE HISTORY dev.bronze.sales

In [0]:
UPDATE delta.`/Volumes/dev/bronze/sales_vol/delta/`
SET Description = "test"
WHERE StockCode == "85123A"
AND Description == "WHITE HANGING HEART T-LIGHT HOLDER"

In [0]:
DELETE FROM dev.bronze.sales
WHERE InvoiceNo = '536367' and StockCode ='84879'


In [0]:
UPDATE dev.bronze.sales
SET Description = "testing change data feed"
WHERE StockCode == "85123A" AND InvoiceNo == "536373"
AND Description == "WHITE HANGING HEART T-LIGHT HOLDER"

In [0]:
INSERT INTO dev.bronze.metadata(table_name, version_processed)
SELECT 'sales', 5

In [0]:
SELECT * FROM dev.bronze.metadata

In [0]:
%python
from delta.tables import DeltaTable
bronze = DeltaTable.forName(spark, "dev.bronze.sales")
start_version = spark.sql(
    f"""
    SELECT COALESCE(MAX(version_processed), 0) + 1 AS v
    FROM dev.bronze.metadata
    WHERE table_name = 'sales'
""").collect()[0][0]

end_version = bronze.history().selectExpr("max(version) as v").collect()[0][0]
print(start_version)
print(end_version)

In [0]:
DELETE FROM  dev.bronze.metadata WHERE metadata_key =1

In [0]:
SELECT *
  FROM table_changes('dev.bronze.sales', 8, 7)
  WHERE _change_type IN ('insert', 'delete', 'update_postimage')

In [0]:
-- SHOW TBLPROPERTIES dev.bronze.sales
-- SHOW CREATE TABLE dev.bronze.sales
-- DESCRIBE DETAIL dev.bronze.sales
DESCRIBE HISTORY dev.bronze.sales

In [0]:
%python
q = f"""
MERGE INTO dev.silver.sales t
USING (
  SELECT *
  FROM table_changes('dev.bronze.sales', {start_version}, {end_version})
  WHERE _change_type IN ('insert', 'delete', 'update_postimage')
) s
ON t.InvoiceNo = s.InvoiceNo AND t.StockCode = s.StockCode
WHEN MATCHED AND s._change_type = 'update_postimage' THEN UPDATE SET *
WHEN MATCHED AND s._change_type = 'delete' THEN DELETE
WHEN NOT MATCHED AND s._change_type = 'insert' THEN INSERT *
"""
print(q)
result = spark.sql(q)
display(result)

In [0]:
merge INTO dev.bronze.metadata AS t
USING (SELECT MAX(version) as version, 'sales' as table_name FROM(DESCRIBE HISTORY dev.bronze.sales)) AS s
ON t.table_name = s.table_name
WHEN MATCHED THEN UPDATE SET version_processed = s.version